Deutsch-Jozsa Algorithm Implementation in Qiskit

In [1]:
# Install qiskit if not already installed
# !pip install qiskit qiskit-aer

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import random

Block 2: Create Query (Query Gate) Functions

In [ ]:
def create_constant_oracle(n):
    """
    Creates a constant oracle that returns either 0 or 1 for all inputs.
    
    Args:
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Query circuit
    """
    oracle = QuantumCircuit(n + 1, name="Constant Query")
    
    # Randomly choose to return 0 (do nothing) or 1 (flip output qubit)
    output = random.randint(0, 1)
    
    if output == 1:
        oracle.x(n)  # Flip the output qubit
    
    return oracle, "constant"


def create_balanced_oracle(n):
    """
    Creates a balanced oracle that returns 0 for half inputs and 1 for other half.
    
    Args:
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Query circuit
    """
    oracle = QuantumCircuit(n + 1, name="Balanced Query")
    
    # Create a random balanced function
    # One simple approach: use CNOT gates controlled by random qubits
    b_str = "".join([random.choice(['0', '1']) for _ in range(n)])
    
    # Apply X gates based on the random binary string
    for i, bit in enumerate(b_str):
        if bit == '1':
            oracle.x(i)
    
    # Apply CNOT gates from each input qubit to output qubit
    for i in range(n):
        oracle.cx(i, n)
    
    # Reverse the X gates
    for i, bit in enumerate(b_str):
        if bit == '1':
            oracle.x(i)
    
    return oracle, "balanced"

Block 3: Create Random Query Function

In [3]:
def create_random_oracle(n):
    """
    Randomly creates either a constant or balanced oracle.
    
    Args:
        n: Number of input qubits
    
    Returns:
        tuple: (QuantumCircuit, oracle_type)
    """
    oracle_type = random.choice(['constant', 'balanced'])
    
    if oracle_type == 'constant':
        return create_constant_oracle(n)
    else:
        return create_balanced_oracle(n)

Block 4: Implement Deutsch-Jozsa Algorithm

In [4]:
def deutsch_jozsa_algorithm(oracle, n):
    """
    Implements the Deutsch-Jozsa algorithm.
    
    Args:
        oracle: Quantum circuit representing the query gate
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Complete Deutsch-Jozsa circuit
    """
    # Create quantum and classical registers
    qr = QuantumRegister(n + 1, 'q')
    cr = ClassicalRegister(n, 'c')
    dj_circuit = QuantumCircuit(qr, cr)
    
    # Step 1: Initialize the output qubit to |1⟩
    dj_circuit.x(n)
    
    # Step 2: Apply Hadamard gates to all qubits
    for i in range(n + 1):
        dj_circuit.h(i)
    
    dj_circuit.barrier()
    
    # Step 3: Apply the oracle
    dj_circuit.compose(oracle, inplace=True)
    
    dj_circuit.barrier()
    
    # Step 4: Apply Hadamard gates to input qubits
    for i in range(n):
        dj_circuit.h(i)
    
    dj_circuit.barrier()
    
    # Step 5: Measure the input qubits
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

Block 5: Function to Run and Analyze Results

In [6]:
def run_deutsch_jozsa(circuit, shots=1024):
    """
    Runs the Deutsch-Jozsa circuit and returns results.
    
    Args:
        circuit: Deutsch-Jozsa quantum circuit
        shots: Number of measurement shots
    
    Returns:
        dict: Measurement results
    """
    # Use AerSimulator
    simulator = AerSimulator()
    
    # Execute the circuit
    job = simulator.run(circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    return counts


def interpret_results(counts, n):
    """
    Interprets the measurement results.
    
    Args:
        counts: Measurement results dictionary
        n: Number of input qubits
    
    Returns:
        str: 'constant' or 'balanced'
    """
    # Get the most common measurement result
    most_common = max(counts, key=counts.get)
    
    # If all zeros, function is constant
    if most_common == '0' * n:
        return "constant"
    else:
        return "balanced"

Block 6: Main Execution and Testing

In [13]:
def test_deutsch_jozsa(n=3):
    """
    Complete test of the Deutsch-Jozsa algorithm.
    
    Args:
        n: Number of input qubits (default: 3)
    """
    print(f"=" * 60)
    print(f"Testing Deutsch-Jozsa Algorithm with {n} input qubits")
    print(f"=" * 60)
    
    # Create a random oracle
    oracle, actual_type = create_random_oracle(n)
    
    print(f"\nActual Query Type: {actual_type}")
    print(f"\nQuery Circuit:")
    print(oracle.draw(output='text'))
    
    # Create the Deutsch-Jozsa circuit
    dj_circuit = deutsch_jozsa_algorithm(oracle, n)
    
    print(f"\nComplete Deutsch-Jozsa Circuit:")
    print(dj_circuit.draw(output='mpl'))
    
    # Run the circuit
    counts = run_deutsch_jozsa(dj_circuit)
    
    print(f"\nMeasurement Results:")
    print(counts)
    
    # Interpret results
    predicted_type = interpret_results(counts, n)
    
    print(f"\nPredicted Query Type: {predicted_type}")
    print(f"Actual Query Type: {actual_type}")
    
    if predicted_type == actual_type:
        print("✓ SUCCESS: Prediction matches actual query type!")
    else:
        print("✗ FAILED: Prediction does not match actual query type!")
    
    print(f"=" * 60)
    
    return dj_circuit, oracle, counts


# Run the test
if __name__ == "__main__":
    circuit, oracle, results = test_deutsch_jozsa(n=3)

Testing Deutsch-Jozsa Algorithm with 3 input qubits

Actual Query Type: balanced

Query Circuit:
                    
q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘

Complete Deutsch-Jozsa Circuit:
Figure(1123.61x451.5)

Measurement Results:
{'111': 1024}

Predicted Query Type: balanced
Actual Query Type: balanced
✓ SUCCESS: Prediction matches actual query type!


In [ ]:
"""
Lab 8: Implementation of Deutsch's Algorithm
Simple and concise implementation
"""

import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

print("="*60)
print("DEUTSCH'S ALGORITHM - 2 QUBIT CASE")
print("="*60)

# =============================================================================
# STEP 1: Define the 4 Possible Query Gates (Oracles)
# =============================================================================

def deutsch_function(case):
    """
    Create oracle for one of 4 possible functions:
    case 0: f(x) = 0 (constant)
    case 1: f(x) = 1 (constant)
    case 2: f(x) = x (balanced)
    case 3: f(x) = NOT x (balanced)
    """
    oracle = QuantumCircuit(2, name=f'Oracle_{case}')
    
    if case == 0:
        # f(x) = 0: Do nothing
        pass
    elif case == 1:
        # f(x) = 1: Flip output
        oracle.x(1)
    elif case == 2:
        # f(x) = x: CNOT
        oracle.cx(0, 1)
    elif case == 3:
        # f(x) = NOT x: X then CNOT
        oracle.x(0)
        oracle.cx(0, 1)
        oracle.x(0)
    
    return oracle

# =============================================================================
# STEP 2: Create Complete Deutsch Circuit
# =============================================================================

def deutsch_algorithm(oracle):
    """Build complete Deutsch's algorithm circuit"""
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(1, 'c')
    circuit = QuantumCircuit(qr, cr)
    
    # Initialize: |0⟩|1⟩
    circuit.x(1)
    circuit.barrier()
    
    # Apply Hadamard to both qubits
    circuit.h(0)
    circuit.h(1)
    circuit.barrier()
    
    # Apply oracle
    circuit.compose(oracle, inplace=True)
    circuit.barrier()
    
    # Apply Hadamard to input qubit
    circuit.h(0)
    circuit.barrier()
    
    # Measure input qubit
    circuit.measure(0, 0)
    
    return circuit

# =============================================================================
# STEP 3: Run and Determine Result
# =============================================================================

def run_deutsch(case):
    """
    Run Deutsch's algorithm and return result
    Returns: 'constant' or 'balanced'
    """
    # Create oracle
    oracle = deutsch_function(case)
    
    # Build circuit
    circuit = deutsch_algorithm(oracle)
    
    # Run simulation
    simulator = AerSimulator()
    compiled = transpile(circuit, simulator)
    job = simulator.run(compiled, shots=1)
    result = job.result()
    counts = result.get_counts()
    
    # Determine result
    measured = list(counts.keys())[0]
    if measured == '0':
        return 'constant', circuit, counts
    else:
        return 'balanced', circuit, counts

# =============================================================================
# STEP 4: Test All 4 Cases
# =============================================================================

print("\n" + "="*60)
print("TESTING ALL 4 DEUTSCH FUNCTIONS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for case in range(4):
    result, circuit, counts = run_deutsch(case)
    
    # Determine actual type
    actual = 'constant' if case < 2 else 'balanced'
    
    # Print result
    print(f"\nCase {case}: f(x) = ", end="")
    if case == 0:
        print("0 (constant)")
    elif case == 1:
        print("1 (constant)")
    elif case == 2:
        print("x (balanced)")
    else:
        print("NOT x (balanced)")
    
    print(f"  Result: {result} ✓" if result == actual else f"  Result: {result} ✗")
    print(f"  Measurement: {list(counts.keys())[0]}")
    
    # Draw circuit
    circuit.draw('mpl', ax=axes[case], style='iqp')
    axes[case].set_title(f'Case {case}: {result.upper()}', 
                         fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('deutsch_all_cases.png', dpi=300, bbox_inches='tight')
print("\n💾 Circuits saved as 'deutsch_all_cases.png'")
plt.show()

# =============================================================================
# STEP 5: Detailed View of One Random Case
# =============================================================================

print("\n" + "="*60)
print("DETAILED VIEW: RANDOM FUNCTION")
print("="*60)

# Pick random case
random_case = np.random.randint(0, 4)
print(f"\n🎲 Selected Case: {random_case}")

# Get oracle and result
oracle = deutsch_function(random_case)
result, circuit, counts = run_deutsch(random_case)

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Oracle circuit
oracle.draw('mpl', ax=axes[0], style='iqp')
axes[0].set_title(f'Query Gate (Oracle) - Case {random_case}', 
                  fontsize=12, fontweight='bold')

# Complete circuit
circuit.draw('mpl', ax=axes[1], style='iqp')
axes[1].set_title(f"Complete Deutsch Circuit\nResult: {result.upper()}", 
                  fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('deutsch_detailed.png', dpi=300, bbox_inches='tight')
print(f"\n💾 Detailed view saved as 'deutsch_detailed.png'")
plt.show()

# Summary
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Function type: {result}")
print(f"Measured state: {list(counts.keys())[0]}")
print(f"Classical queries needed: 2")
print(f"Quantum queries needed: 1")
print(f"Speedup: 2x")
print("="*60)